In [1]:
import pandas as pd

# читаем все файлы
drivers = pd.read_csv('drivers.csv')
vehicles = pd.read_csv('vehicles.csv')
insurance = pd.read_csv('insurance.csv')
driver_vehicle_intervals = pd.read_csv('driver_vehicle_intervals.csv', parse_dates=['start_time', 'end_time'])
telemetry = pd.read_csv('telemetry.csv', parse_dates=['timestamp'])
epd_events = pd.read_csv('epd_events.csv', parse_dates=['date'])

# 1. Сначала связываем телеметрию с интервалами
merged = telemetry.merge(
    driver_vehicle_intervals,
    on='vehicle_id',
    how='left'
)

# 2. Фильтруем только те строки, где timestamp попадает в интервал
merged = merged[
    (merged['timestamp'] >= merged['start_time']) &
    (merged['timestamp'] <= merged['end_time'])
]

# 3. Подтягиваем информацию о водителях
merged = merged.merge(
    drivers,
    on='driver_id',
    how='left'
)

# 4. Подтягиваем данные о ТС
merged = merged.merge(
    vehicles,
    on='vehicle_id',
    how='left'
)

# 5. Подтягиваем страховку (vehicle_id)
merged = merged.merge(
    insurance,
    on='vehicle_id',
    how='left'
)

# 6. Подтягиваем события ЭПД по epd_doc_id
merged = merged.merge(
    epd_events,
    left_on='epd_doc_id',  # из driver_vehicle_intervals
    right_on='doc_id',
    how='left'
)

# готовый объединённый датафрейм:
print(merged.head())


  telemetry_id           timestamp vehicle_id_x  speed_kmh  accel_m_s2  \
0      T000001 2025-09-22 14:00:00         V002      56.48       0.032   
1      T000002 2025-09-22 14:00:20         V002      59.64      -0.319   
2      T000003 2025-09-22 14:00:40         V002      61.14      -0.237   
3      T000004 2025-09-22 14:01:00         V002      67.98      -0.232   
4      T000005 2025-09-22 14:01:20         V002      58.32      -0.475   

    event        lat       lon  heading_deg interval_id  ... provider  \
0  normal  52.370514  4.895287         2.97       I0001  ...       NN   
1  normal  52.370494  4.895555         6.75       I0001  ...       NN   
2  normal  52.370660  4.895850         7.28       I0001  ...       NN   
3  normal  52.370722  4.895882         7.71       I0001  ...       NN   
4  normal  52.370724  4.896067         5.21       I0001  ...       NN   

   start_date    end_date tariff_class deductible_eur annual_premium_eur  \
0  2025-07-08  2026-07-08            A  

In [2]:
merged

,telemetry_id,timestamp,vehicle_id_x,speed_kmh,accel_m_s2,event,lat,lon,heading_deg,interval_id,...,provider,start_date,end_date,tariff_class,deductible_eur,annual_premium_eur,doc_id,date,driver_id_y,vehicle_id_y
0,T000001,2025-09-22 14:00:00,V002,56.48,0.032,normal,52.370514,4.895287,2.97,I0001,...,NN,2025-07-08,2026-07-08,A,1500,4963,DOC-0020,2025-09-21,D008,V002
1,T000002,2025-09-22 14:00:20,V002,59.64,-0.319,normal,52.370494,4.895555,6.75,I0001,...,NN,2025-07-08,2026-07-08,A,1500,4963,DOC-0020,2025-09-21,D008,V002
2,T000003,2025-09-22 14:00:40,V002,61.14,-0.237,normal,52.370660,4.895850,7.28,I0001,...,NN,2025-07-08,2026-07-08,A,1500,4963,DOC-0020,2025-09-21,D008,V002
3,T000004,2025-09-22 14:01:00,V002,67.98,-0.232,normal,52.370722,4.895882,7.71,I0001,...,NN,2025-07-08,2026-07-08,A,1500,4963,DOC-0020,2025-09-21,D008,V002
4,T000005,2025-09-22 14:01:20,V002,58.32,-0.475,normal,52.370724,4.896067,5.21,I0001,...,NN,2025-07-08,2026-07-08,A,1500,4963,DOC-0020,2025-09-21,D008,V002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43086,T024103,2025-09-20 08:49:10,V005,78.60,-0.845,normal,52.418164,4.966854,298.20,I0030,...,Aegon,2025-08-16,2026-08-16,B,500,3107,DOC-0036,2025-09-21,D005,V005
43087,T024104,2025-09-20 08:49:20,V005,63.70,0.223,normal,52.418134,4.966987,300.39,I0030,...,Aegon,2025-08-16,2026-08-16,B,500,3107,DOC-0036,2025-09-21,D005,V005
43088,T024105,2025-09-20 08:49:30,V005,64.86,0.049,normal,52.418139,4.967194,300.02,I0030,...,Aegon,2025-08-16,2026-08-16,B,500,3107,DOC-0036,2025-09-21,D005,V005
43089,T024106,2025-09-20 08:49:40,V005,44.20,0.228,normal,52.418349,4.967524,298.40,I0030,...,Aegon,2025-08-16,2026-08-16,B,500,3107,DOC-0036,2025-09-21,D005,V005


In [3]:
merged.to_csv("/Users/stepanbutakov/Documents/питон/pythonProject/хахатон/hackathon_testdata_final/merged_telemetry_intervals.csv", index=False)

In [4]:
import pandas as pd

# 1. Загружаем объединенный файл
# (он у тебя уже сформирован после merge driver_vehicle_intervals + telemetry + drivers + vehicles + insurance + epd)
df = pd.read_csv(
    'merged_telemetry_intervals.csv',  # путь к твоему файлу
    parse_dates=['timestamp', 'start_time', 'end_time'], 
    low_memory=False
)

# 2. Берём driver_id из driver_vehicle_intervals (колонка driver_id_x)
df['driver_id'] = df['driver_id_x']

# 3. Назначаем веса за рискованные события
weights = {
    'hard_brake': 3,   # резкое торможение
    'hard_accel': 3,   # резкий разгон
    'lane_change': 1   # перестроение
}

# 4. Считаем баллы риска для каждой записи
df['risk_points'] = df['event'].map(weights).fillna(0)

# 5. Группируем по водителю: считаем количество записей и сумму баллов риска
agg = df.groupby('driver_id').agg(
    total_events=('telemetry_id', 'count'),
    total_risk_points=('risk_points', 'sum')
).reset_index()

# 6. Рассчитываем средний риск и индекс аккуратности
agg['avg_risk_per_event'] = agg['total_risk_points'] / agg['total_events']
agg['accuracy_index'] = (100 - agg['avg_risk_per_event'] * 10).clip(lower=0)

# 7. Присоединяем имена водителей
agg = agg.merge(
    df[['driver_id', 'name']].drop_duplicates(), 
    on='driver_id', 
    how='left'
)

# 8. Сортируем по индексу аккуратности (чем выше, тем аккуратнее водитель)
agg_sorted = agg.sort_values('accuracy_index', ascending=False)

# 9. Выводим топ-10 аккуратных водителей
print("Топ-10 водителей по индексу аккуратности:")
print(agg_sorted[['driver_id', 'name', 'accuracy_index']].head(10))



Топ-10 водителей по индексу аккуратности:
  driver_id      name  accuracy_index
3      D004  Driver_4       97.917412
7      D008  Driver_8       97.856880
1      D002  Driver_2       97.838542
4      D005  Driver_5       97.806725
6      D007  Driver_7       97.799464
0      D001  Driver_1       97.778667
2      D003  Driver_3       97.774608
5      D006  Driver_6       97.594011


In [5]:
df = pd.read_csv('merged_telemetry_intervals.csv')
df

,telemetry_id,timestamp,vehicle_id_x,speed_kmh,accel_m_s2,event,lat,lon,heading_deg,interval_id,...,provider,start_date,end_date,tariff_class,deductible_eur,annual_premium_eur,doc_id,date,driver_id_y,vehicle_id_y
0,T000001,2025-09-22 14:00:00,V002,56.48,0.032,normal,52.370514,4.895287,2.97,I0001,...,NN,2025-07-08,2026-07-08,A,1500,4963,DOC-0020,2025-09-21,D008,V002
1,T000002,2025-09-22 14:00:20,V002,59.64,-0.319,normal,52.370494,4.895555,6.75,I0001,...,NN,2025-07-08,2026-07-08,A,1500,4963,DOC-0020,2025-09-21,D008,V002
2,T000003,2025-09-22 14:00:40,V002,61.14,-0.237,normal,52.370660,4.895850,7.28,I0001,...,NN,2025-07-08,2026-07-08,A,1500,4963,DOC-0020,2025-09-21,D008,V002
3,T000004,2025-09-22 14:01:00,V002,67.98,-0.232,normal,52.370722,4.895882,7.71,I0001,...,NN,2025-07-08,2026-07-08,A,1500,4963,DOC-0020,2025-09-21,D008,V002
4,T000005,2025-09-22 14:01:20,V002,58.32,-0.475,normal,52.370724,4.896067,5.21,I0001,...,NN,2025-07-08,2026-07-08,A,1500,4963,DOC-0020,2025-09-21,D008,V002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43086,T024103,2025-09-20 08:49:10,V005,78.60,-0.845,normal,52.418164,4.966854,298.20,I0030,...,Aegon,2025-08-16,2026-08-16,B,500,3107,DOC-0036,2025-09-21,D005,V005
43087,T024104,2025-09-20 08:49:20,V005,63.70,0.223,normal,52.418134,4.966987,300.39,I0030,...,Aegon,2025-08-16,2026-08-16,B,500,3107,DOC-0036,2025-09-21,D005,V005
43088,T024105,2025-09-20 08:49:30,V005,64.86,0.049,normal,52.418139,4.967194,300.02,I0030,...,Aegon,2025-08-16,2026-08-16,B,500,3107,DOC-0036,2025-09-21,D005,V005
43089,T024106,2025-09-20 08:49:40,V005,44.20,0.228,normal,52.418349,4.967524,298.40,I0030,...,Aegon,2025-08-16,2026-08-16,B,500,3107,DOC-0036,2025-09-21,D005,V005


In [6]:
import plotly.express as px

fig = px.scatter_geo(
    df[df['interval_id'] == 'I0001'],
    lat = 'lat',
    lon = 'lon',
    color='interval_id',
    color_continuous_scale=px.colors.sequential.Viridis,  # цветовая схема
    size_max=15
)

In [7]:
fig

In [8]:
#!/usr/bin/env python3
"""
insurance_recommendation_pipeline.py

Pipeline для:
- агрегации телеметрии по парам (driver_id, vehicle_id),
- вычисления прокси risk_score (эвристика),
- кластеризации (низкий/средний/высокий риск),
- расчёта рекомендованной скидки и простых правил по франшизе,
- подбора совместимости driver <-> vehicle.

Выход: /mnt/data/recommendations.csv
"""
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

INPUT_PATH = "merged_telemetry_intervals.csv"
OUT_CSV = "recommendations.csv"

def load_csv_try(path):
    return pd.read_csv(path, low_memory=False) if os.path.exists(path) else None

def main(input_path=INPUT_PATH, out_csv=OUT_CSV):
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"Input file not found: {input_path}")

    df = pd.read_csv(input_path, low_memory=False)

    # стандартизируем имена колонок driver_id / vehicle_id (есть варианты merged с _x/_y)
    if "driver_id" not in df.columns:
        if "driver_id_x" in df.columns:
            df["driver_id"] = df["driver_id_x"]
        elif "driver_id_y" in df.columns:
            df["driver_id"] = df["driver_id_y"]
    if "vehicle_id" not in df.columns:
        if "vehicle_id_x" in df.columns:
            df["vehicle_id"] = df["vehicle_id_x"]
        elif "vehicle_id_y" in df.columns:
            df["vehicle_id"] = df["vehicle_id_y"]

    # Проверка
    if "driver_id" not in df.columns or "vehicle_id" not in df.columns:
        raise ValueError("Ожидаются колонки driver_id и vehicle_id в input CSV")

    # Преобразование дат/чисел
    for col in ["timestamp", "start_time", "end_time", "date", "hired_date"]:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors="coerce")

    for col in ["speed_kmh", "accel_m_s2", "odometer_km", "annual_premium_eur", "deductible_eur"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)

    # Вычислим длительность поездки, если есть start_time/end_time
    if "start_time" in df.columns and "end_time" in df.columns:
        df["trip_seconds"] = (pd.to_datetime(df["end_time"], errors="coerce") - pd.to_datetime(df["start_time"], errors="coerce")).dt.total_seconds().fillna(0)
    else:
        df["trip_seconds"] = 0

    # Группировка по driver_id + vehicle_id
    group_cols = ["driver_id", "vehicle_id"]
    agg = df.groupby(group_cols).agg(
        trips=("trip_seconds", "count"),
        total_trip_seconds=("trip_seconds", "sum"),
        avg_speed_kmh=("speed_kmh", "mean"),
        max_speed_kmh=("speed_kmh", "max"),
        std_speed_kmh=("speed_kmh", "std"),
        avg_accel=("accel_m_s2", "mean"),
        max_accel=("accel_m_s2", "max"),
        std_accel=("accel_m_s2", "std")
    ).reset_index().fillna(0)

    # Считаем типы событий (hard_brake, hard_accel, lane_change, normal)
    def count_events(s):
        s = s.fillna("normal")
        c = s.value_counts()
        return pd.Series({
            "hard_brake_count": int(c.get("hard_brake", 0)),
            "hard_accel_count": int(c.get("hard_accel", 0)),
            "lane_change_count": int(c.get("lane_change", 0)),
            "normal_count": int(c.get("normal", 0))
        })

    if "event" in df.columns:
        events = df.groupby(group_cols)["event"].apply(count_events).reset_index()
    else:
        # если нет колонки event — ставим 0
        events = agg[["driver_id", "vehicle_id"]].copy()
        events["hard_brake_count"] = 0
        events["hard_accel_count"] = 0
        events["lane_change_count"] = 0
        events["normal_count"] = 0

    features = agg.merge(events, on=group_cols, how="left").fillna(0)

    # Попробуем получить odometer/vehicle_age/tenure из объединённого файла, если есть
    if "odometer_km" in df.columns:
        veh_od = df[["vehicle_id", "odometer_km"]].drop_duplicates(subset=["vehicle_id"])
        features = features.merge(veh_od, on="vehicle_id", how="left")
    else:
        features["odometer_km"] = 0

    if "year" in df.columns:
        veh_year = df[["vehicle_id", "year"]].drop_duplicates(subset=["vehicle_id"])
        features = features.merge(veh_year, on="vehicle_id", how="left")
        features["vehicle_age"] = pd.Timestamp.now().year - pd.to_numeric(features["year"], errors="coerce")
    else:
        features["vehicle_age"] = 0

    if "hired_date" in df.columns:
        drv = df[["driver_id", "hired_date"]].drop_duplicates(subset=["driver_id"])
        drv["tenure_days"] = (pd.Timestamp.now() - pd.to_datetime(drv["hired_date"], errors="coerce")).dt.days.fillna(0)
        features = features.merge(drv[["driver_id", "tenure_days"]], on="driver_id", how="left")
    else:
        features["tenure_days"] = 0

    # Заполняем NaN для числовых
    num_cols = features.select_dtypes(include=[np.number]).columns
    features[num_cols] = features[num_cols].fillna(0)

    # Соберём признаки для proxy-risk
    score_cols = [
        "hard_brake_count",
        "hard_accel_count",
        "lane_change_count",
        "avg_speed_kmh",
        "avg_accel",
        "vehicle_age",
        "odometer_km",
        "tenure_days"
    ]
    available = [c for c in score_cols if c in features.columns]

    # Стандартизация
    scaler = StandardScaler()
    X = scaler.fit_transform(features[available])

    # Веса: tenure_days уменьшает риск (отрицательный вес), остальные увеличивают
    weights = np.array([ -1.0 if c == "tenure_days" else 1.0 for c in available ], dtype=float)

    # Вычисляем сырое значение риска и нормируем в [0,1]
    risk_raw = (X * weights).sum(axis=1)
    risk_score = (risk_raw - risk_raw.min()) / (risk_raw.max() - risk_raw.min() + 1e-9)
    features["risk_score"] = risk_score

    # Кластеризация в 3 группы (низкий/средний/высокий)
    kmeans = KMeans(n_clusters=3, random_state=42)
    features["risk_cluster"] = kmeans.fit_predict(features[available])

    # Рекомендация скидки (эвристика)
    base_discount = 0.10        # 10%
    max_extra = 0.10            # до +10% для безопасных
    features["recommended_discount_pct"] = (base_discount + (1 - features["risk_score"]) * max_extra).clip(0, 0.30)

    # Правила по страховке (эвристика)
    def insurance_reco(r):
        if r < 0.33:
            return ("low", "increase_discount", -50)
        elif r > 0.66:
            return ("high", "increase_deductible", 100)
        else:
            return ("medium", "standard", 0)

    features[["risk_bucket", "suggested_action", "suggested_deductible_adj_eur"]] = features["risk_score"].apply(
        lambda x: pd.Series(insurance_reco(x))
    )

    # Совместимость driver <-> vehicle:
    drv_vec = features.groupby("driver_id")[available].mean().fillna(0)
    veh_vec = features.groupby("vehicle_id")[available].mean().fillna(0)

    # Нормализация перед косинусной схожестью
    if len(drv_vec) == 0 or len(veh_vec) == 0:
        # Защита на случай пустых данных
        features["best_vehicle_id"] = None
        features["compatibility"] = 0.0
    else:
        drv_norm = pd.DataFrame(StandardScaler().fit_transform(drv_vec), index=drv_vec.index, columns=drv_vec.columns)
        veh_norm = pd.DataFrame(StandardScaler().fit_transform(veh_vec), index=veh_vec.index, columns=veh_vec.columns)
        sim = cosine_similarity(drv_norm, veh_norm)
        sim_df = pd.DataFrame(sim, index=drv_norm.index, columns=veh_norm.index)

        # Для каждого водителя выбираем лучшую машину
        best = []
        for d in sim_df.index:
            best_vehicle = sim_df.loc[d].idxmax()
            best_score = float(sim_df.loc[d].max())
            best.append({"driver_id": d, "best_vehicle_id": best_vehicle, "compatibility": best_score})
        best_df = pd.DataFrame(best)
        features = features.merge(best_df, on="driver_id", how="left")

    # Подготовка и сохранение выходного файла
    out_cols = [
        "driver_id", "vehicle_id", "trips", "total_trip_seconds",
        "avg_speed_kmh", "max_speed_kmh", "avg_accel", "max_accel",
        "hard_brake_count", "hard_accel_count", "lane_change_count",
        "risk_score", "risk_cluster", "recommended_discount_pct",
        "risk_bucket", "suggested_action", "suggested_deductible_adj_eur",
        "best_vehicle_id", "compatibility"
    ]
    out_cols = [c for c in out_cols if c in features.columns]
    out_df = features[out_cols].copy()

    # Красивое представление скидки в процентах (опционально)
    if "recommended_discount_pct" in out_df.columns:
        out_df["recommended_discount_pct"] = (out_df["recommended_discount_pct"] * 100).round(2)

    out_df.to_csv(out_csv, index=False)
    print(f"Saved recommendations to: {out_csv}")
    # Вывод топ-10 самых безопасных (по risk_score)
    print("Top 10 (lowest risk_score):")
    print(out_df.drop_duplicates().sort_values("risk_score").to_string(index=False))

if __name__ == "__main__":
    main()


Saved recommendations to: recommendations.csv
Top 10 (lowest risk_score):
driver_id vehicle_id  trips  total_trip_seconds  avg_speed_kmh  max_speed_kmh  avg_accel  max_accel  risk_score  risk_cluster  recommended_discount_pct risk_bucket    suggested_action  suggested_deductible_adj_eur best_vehicle_id  compatibility
     D005       V001    706          11903160.0      58.862139          98.92  -0.020916      3.566    0.000000             0                     20.00         low   increase_discount                           -50            V005       0.431620
     D004       V004    498           4960080.0      59.148032          97.20  -0.007329      3.340    0.089672             0                     19.10         low   increase_discount                           -50            V002       0.773642
     D008       V001   1003          16542360.0      58.689143          98.92  -0.031431      3.605    0.203162             0                     17.97         low   increase_discount        